[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

# Pytorch - Redes Neuronales

In [ ]:
import torch

## Modelos secuenciales

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [ ]:
from sklearn.datasets import fetch_openml

# descarga datos

mnist = fetch_openml('mnist_784', version=1)
X, Y = mnist["data"], mnist["target"]

X.shape, Y.shape

((70000, 784), (70000,))

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:
import numpy as np

# normalización y split

X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.int), Y[60000:].astype(np.int)

In [ ]:
type(X_train)

numpy.ndarray

## Optimizadores y Funciones de pérdida

Mientras que los optimizadores se encuentran en el paquete `torch.optim`

## Modelos custom

In [ ]:
# creamos una clase que hereda de `torch.nn.Module`

class Model(torch.nn.Module):
    
    # constructor
    def __init__(self, D_in, H, D_out):
        
        # llamamos al constructor de la clase madre
        super(Model, self).__init__()
        
        # definimos nuestras capas
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)
        
    # lógica para calcular las salidas de la red
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
model = Model(784, 100, 10)
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

In [ ]:
X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

In [ ]:
type(X_t),X_t.dtype

(torch.Tensor, torch.float32)

In [ ]:
def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [ ]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

In [ ]:
model.to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1): 
    
    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())
    
    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()
    
    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")
        
y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 0.25189
Epoch 20/100 Loss 0.24439
Epoch 30/100 Loss 0.23877
Epoch 40/100 Loss 0.23398
Epoch 50/100 Loss 0.22969
Epoch 60/100 Loss 0.22574
Epoch 70/100 Loss 0.22206
Epoch 80/100 Loss 0.21860
Epoch 90/100 Loss 0.21533
Epoch 100/100 Loss 0.21222


0.947